# AutoGen Tool calling capabilities with Unity Catalog

## Prerequisites

**API Key**
To run this tutorial, you will need an OpenAI API key. 

Once you have acquired your key, set it to the environment variable `OPENAI_API_KEY`.

Below, we validate that this key is set properly in your environment.

**Packages**

To interface with both UnityCatalog and AutoGen, you will need to install the following package:

```shell
pip install unitycatalog-autogen
```

**Note**
The official Microsoft AutoGen package has been renamed from `pyautogen` to `autogen-agentchat`. 
There are additional forked version of the AutoGen package that are not contributed by Microsoft and will not work with this integration. 
For further information, please see the [official clarification statement](https://github.com/microsoft/autogen/discussions/4217). 

**Note**
The example shown here is for `autogen` 0.4.0 and above. Earlier versions of the library will not function with the Unity Catalog AI Toolkit.

In [1]:
import os

assert "OPENAI_API_KEY" in os.environ, (
    "Please set the OPENAI_API_KEY environment variable to your OpenAI API key"
)

## Configuration and Client setup

In order to connect to your Unity Catalog server, you'll need an instance of the `ApiClient` from the `unitycatalog-client` package. 

> Note: If you don't already have a Catalog and a Schema created, be sure to create them before running this notebook and adjust the `CATALOG` and `SCHEMA` variables below to suit.

In [2]:
from unitycatalog.ai.autogen.toolkit import UCFunctionToolkit
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration

In [3]:
config = Configuration()
config.host = "http://localhost:8080/api/2.1/unity-catalog"

# The base ApiClient is async
api_client = ApiClient(configuration=config)

client = UnitycatalogFunctionClient(api_client=api_client)

CATALOG = "AICatalog"
SCHEMA = "AISchema"

## Define a function and register it to Unity Catalog

In this next section, we'll be defining some Python functions and creating them within Unity Catalog so that they can be retrieved and used as tools by an AutoGen agent. 

There are a few things to keep in mind when creating functions for use with the `create_python_function` API:

- Ensure that your have properly defined types for all arguments and for the return of the function.
- Ensure that you have a Google-style docstring defined that includes descriptions for the function, each argument, and the return of the function. This is critical, as these are used to populate the metadata associated with the function within Unity Catalog, providing contextual data for an LLM to understand when and how to call the tool associated with this function.
- If there are packages being called that are not part of core Python, ensure that the import statements are locally scoped (defined within the function body).

In [4]:
def calculate_vpd(temperature_c: float, dew_point_c: float) -> float:
    """
    Calculate Vapor Pressure Deficit (VPD) given temperature and dew point in Celsius.

    Args:
        temperature_c (float): Air temperature in Celsius.
        dew_point_c (float): Dew point temperature in Celsius.

    Returns:
        Vapor Pressure Deficit in hPa.
    """
    import math  # local imports are needed for execution

    a = 17.625
    b = 243.04

    e_s = 6.1094 * math.exp((a * temperature_c) / (b + temperature_c))
    e_a = 6.1094 * math.exp((a * dew_point_c) / (b + dew_point_c))

    vpd = e_s - e_a
    return vpd


def fahrenheit_to_celsius(fahrenheit: float) -> float:
    """
    Converts temperature from Fahrenheit to Celsius.

    Args:
        fahrenheit (float): Temperature in degrees Fahrenheit.

    Returns:
        float: Temperature in degrees Celsius.
    """
    return (fahrenheit - 32) * 5.0 / 9.0

In [5]:
client.create_python_function(func=calculate_vpd, catalog=CATALOG, schema=SCHEMA, replace=True)

FunctionInfo(name='calculate_vpd', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='temperature_c', type_text='DOUBLE', type_json='{"name": "temperature_c", "type": "double", "nullable": false, "metadata": {"comment": "Air temperature in Celsius."}}', type_name=<ColumnTypeName.DOUBLE: 'DOUBLE'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='Air temperature in Celsius.'), FunctionParameterInfo(name='dew_point_c', type_text='DOUBLE', type_json='{"name": "dew_point_c", "type": "double", "nullable": false, "metadata": {"comment": "Dew point temperature in Celsius."}}', type_name=<ColumnTypeName.DOUBLE: 'DOUBLE'>, type_precision=None, type_scale=None, type_interval_type=None, position=1, parameter_mode=None, parameter_type=None, parameter_default=None, comment='Dew point temperature in Celsius.')]), data_type=<Co

In [6]:
client.create_python_function(
    func=fahrenheit_to_celsius, catalog=CATALOG, schema=SCHEMA, replace=True
)

FunctionInfo(name='fahrenheit_to_celsius', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='fahrenheit', type_text='DOUBLE', type_json='{"name": "fahrenheit", "type": "double", "nullable": false, "metadata": {"comment": "Temperature in degrees Fahrenheit."}}', type_name=<ColumnTypeName.DOUBLE: 'DOUBLE'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='Temperature in degrees Fahrenheit.')]), data_type=<ColumnTypeName.DOUBLE: 'DOUBLE'>, full_data_type='DOUBLE', return_params=None, routine_body='EXTERNAL', routine_definition='return (fahrenheit - 32) * 5.0 / 9.0', routine_dependencies=None, parameter_style='S', is_deterministic=True, sql_data_access='NO_SQL', is_null_call=False, security_type='DEFINER', specific_name='fahrenheit_to_celsius', comment='Converts temperature from Fahrenheit to Celsius.', properties='

## Create a Toolkit instance of the function(s)

Now that the function has been created within Unity Catalog, we can use the `unitycatalog-autogen` package to create a toolkit instance that our Agent will 'understand' as a valid tool in its APIs. 

In [7]:
vpd_tool = f"{CATALOG}.{SCHEMA}.calculate_vpd"
f_to_c_tool = f"{CATALOG}.{SCHEMA}.fahrenheit_to_celsius"

toolkit = UCFunctionToolkit(function_names=[vpd_tool, f_to_c_tool], client=client)

tools = toolkit.tools

In [8]:
tools

## Create a Conversable Agent that uses our tool

Now we get to actually create an Agent. As part of our definition, we'll be applying the tool defintion from our Toolkit instance. 

In [9]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=OPENAI_API_KEY,
    seed=222,
    temperature=0,
)

weather_agent = AssistantAgent(
    name="assistant",
    system_message="You are a helpful AI assistant that specializes in answering questions about weather phenomena. "
    "If there are tools available to perform these calculations, please use them and ensure that you are operating "
    "with validated data calculations.",
    model_client=model_client,
    tools=tools,
    reflect_on_tool_use=False,
)

## Ask the Agent a question

Now that we have everything configured, let's test out our Agent! 

In [11]:
user_input = "I need some help converting 973.2F to Celsius."

response = await weather_agent.on_messages(
    [TextMessage(content=user_input, source="User")], CancellationToken()
)

response.chat_message

TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=286, completion_tokens=14), content='973.2°F is approximately 522.89°C.', type='TextMessage')